In [1]:
import json
import multiprocessing as mp
import os
import sys

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gc_utils import iteration_name, snapshot_name  # type: ignore
from scipy.interpolate import interp1d

In [21]:
# Save to a JSON file
evolve_mass_loss = 0.45

it_min = 0
it_max = 100
# it_max = 100

sim = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
save_dir = "../data/gc_mass_data/"
save_dir = "/Users/z5114326/Documents/GitHub/gc_statistics/data/mass_data/"

mass_file = save_dir + sim + "_gc_mass_data.hdf5"  # save location
mass_data = proc_data = h5py.File(mass_file, "r")  # open processed data file

In [6]:
mass_data.keys()

<KeysViewHDF5 ['it000']>

In [10]:
def time_bounds(mass_data):
    # the min for both these time frames will be 0
    time_max = 0
    form_time_max = 0

    acc_time_min = 0
    acc_time_max = 0

    for it_id in mass_data.keys():
        # it_id = iteration_name(it)

        # data_file = save_dir + "/" + it_id + "_gc_mass_data.json"

        # # open json file as dict
        # with open(data_file, "r") as file:
        #     mass_dict = json.load(file)

        for gc_id in mass_data[it_id].keys():
            it_time_max = np.max(mass_data[it_id][gc_id]["time"])
            time_max = np.max([time_max, it_time_max])

            it_form_time_max = np.max(mass_data[it_id][gc_id]["form_time"])
            form_time_max = np.max([form_time_max, it_form_time_max])

            if mass_data[it_id][gc_id]["type_flag"][()] > 1:
                it_acc_time_min = np.min(mass_data[it_id][gc_id]["acc_time"])
                it_acc_time_max = np.max(mass_data[it_id][gc_id]["acc_time"])

                acc_time_min = np.min([acc_time_min, it_acc_time_min])
                acc_time_max = np.max([acc_time_max, it_acc_time_max])

    time_bounds_dict = {
        "time": [0, time_max],
        "form_time": [0, form_time_max],
        "acc_time": [acc_time_min, acc_time_max],
    }

    return time_bounds_dict

In [14]:
def create_time_dict(mass_data, num_step=1000):
    time_bounds_dict = time_bounds(mass_data)

    time_grid = np.linspace(time_bounds_dict["time"][0], time_bounds_dict["time"][1], num_step)
    form_time_grid = np.linspace(time_bounds_dict["form_time"][0], time_bounds_dict["form_time"][1], num_step)
    acc_time_grid = np.linspace(time_bounds_dict["acc_time"][0], time_bounds_dict["acc_time"][1], num_step)

    ####################################
    time_dict = {}

    time_dict["time"] = time_grid
    time_dict["form_time"] = form_time_grid
    time_dict["acc_time"] = acc_time_grid

    ####################################

    return time_dict

In [22]:
def time_interpolate(x, y, common_grid):
    interp_func = interp1d(x, y, kind="nearest", bounds_error=False, fill_value=(0, 0))
    # interp_func = interp1d(x, y, kind="previous", bounds_error=False, fill_value=(0, 0))
    return interp_func(common_grid)

In [15]:
time_dict = create_time_dict(mass_data, num_step=1000)

In [23]:
def get_mass_by_formation(it, sim, save_dir, time_dict):
    mass_file = save_dir + sim + "_gc_mass_data.hdf5"  # save location
    mass_data = h5py.File(mass_file, "r")  # open processed data file

    it_id = iteration_name(it)

    print(it_id)

    # for it_id in mass_dict.keys():
    # data_file = save_dir + "/" + it_id + "_gc_mass_data.json"
    # # open json file as dict
    # with open(data_file, "r") as file:
    #     mass_dict = json.load(file)

    # print("make plot dict")

    it_type_dict = {}

    it_type_dict["0"] = {}
    it_type_dict["1"] = {}
    it_type_dict["2"] = {}
    it_type_dict["3"] = {}
    it_type_dict["4"] = {}

    # set up dictionary
    for type_flag_val in range(0, 5):
        it_type_dict[str(type_flag_val)] = {}

        #######################################################################################
        # need to make a mass list for each time frame
        # cosmic time
        time_len = len(time_dict["time"])
        it_type_dict[str(type_flag_val)]["time"] = {}
        it_type_dict[str(type_flag_val)]["time"]["mass"] = np.zeros(time_len)
        it_type_dict[str(type_flag_val)]["time"]["mass_loss"] = np.zeros(time_len)
        it_type_dict[str(type_flag_val)]["time"]["mass_loss_det"] = np.zeros(time_len)

        # formation time
        time_len = len(time_dict["form_time"])
        it_type_dict[str(type_flag_val)]["form_time"] = {}
        it_type_dict[str(type_flag_val)]["form_time"]["mass"] = np.zeros(time_len)
        it_type_dict[str(type_flag_val)]["form_time"]["mass_loss"] = np.zeros(time_len)
        it_type_dict[str(type_flag_val)]["form_time"]["mass_loss_det"] = np.zeros(time_len)

        # accretion time
        if type_flag_val > 1:
            time_len = len(time_dict["acc_time"])
            it_type_dict[str(type_flag_val)]["acc_time"] = {}
            it_type_dict[str(type_flag_val)]["acc_time"]["mass"] = np.zeros(time_len)
            it_type_dict[str(type_flag_val)]["acc_time"]["mass_loss"] = np.zeros(time_len)
            it_type_dict[str(type_flag_val)]["acc_time"]["mass_loss_det"] = np.zeros(time_len)

    #######################################################################################
    for gc_id in mass_data[it_id]:
        type_flag = mass_data[it_id][gc_id]["type_flag"]

        time_lst = mass_data[it_id][gc_id]["time"]
        form_time_lst = mass_data[it_id][gc_id]["form_time"]

        log_mass_lst = mass_data[it_id][gc_id]["log_mass"]

        mass_loss_lst = mass_data[it_id][gc_id]["mass_loss"]
        mass_loss_det_lst = mass_data[it_id][gc_id]["mass_loss_detectable"]

        mass_lst = []
        for log_mass in log_mass_lst:
            if not np.isnan(log_mass):
                mass = 10**log_mass
            else:
                mass = 0
            # if mass == 2130.0:
            #     print(mass)
            mass_lst.append(mass)

        ########################################################################################3
        # cosmic time
        time_mass = time_interpolate(time_lst, mass_lst, time_dict["time"])
        time_mass_loss = time_interpolate(time_lst, mass_loss_lst, time_dict["time"])
        time_mass_loss_det = time_interpolate(time_lst, mass_loss_det_lst, time_dict["time"])

        it_type_dict[str(type_flag)]["time"]["mass"] += time_mass
        it_type_dict[str(type_flag)]["time"]["mass_loss"] += time_mass_loss
        it_type_dict[str(type_flag)]["time"]["mass_loss_det"] += time_mass_loss_det

        # formation time
        form_time_mass = time_interpolate(form_time_lst, mass_lst, time_dict["form_time"])
        form_time_mass_loss = time_interpolate(form_time_lst, mass_loss_lst, time_dict["form_time"])
        form_time_mass_loss_det = time_interpolate(form_time_lst, mass_loss_det_lst, time_dict["form_time"])

        it_type_dict[str(type_flag)]["form_time"]["mass"] += form_time_mass
        it_type_dict[str(type_flag)]["form_time"]["mass_loss"] += form_time_mass_loss
        it_type_dict[str(type_flag)]["form_time"]["mass_loss_det"] += form_time_mass_loss_det

        # accretion time
        if type_flag > 1:
            acc_time_lst = mass_data[it_id][gc_id]["acc_time"]

            acc_time_mass = time_interpolate(acc_time_lst, mass_lst, time_dict["acc_time"])
            acc_time_mass_loss = time_interpolate(acc_time_lst, mass_loss_lst, time_dict["acc_time"])
            acc_time_mass_loss_det = time_interpolate(acc_time_lst, mass_loss_det_lst, time_dict["acc_time"])

            it_type_dict[str(type_flag)]["acc_time"]["mass"] += acc_time_mass
            it_type_dict[str(type_flag)]["acc_time"]["mass_loss"] += acc_time_mass_loss
            it_type_dict[str(type_flag)]["acc_time"]["mass_loss_det"] += acc_time_mass_loss_det

    # close file
    mass_data.close()

    return it_type_dict